In [2]:
import matplotlib.pyplot 
from matplotlib.pyplot import *
import numpy as np
from numpy import arange
from scipy.optimize import linprog
import math

import pulp

from sympy import *

# Matrix Math

Willie W’s Candy makes three types of artisanal chocolate bars: cherry, almond, and raisin. Matrix A gives the amount of ingredients in one batch. Matrix B gives the costs of ingredients from suppliers J and K. Calculate the cost of 100 batches of each candy using ingredients from supplier K.

In [3]:
# ingrediant matrix: sugar, choc, milk
A = np.array([[6,8,1],  # cherry
              [6,4,1],  # almond
              [5,7,1]]) # raisin
 
#  Supplier J is ommitted from the cost matrix
B = np.matrix([[3],  # sugar
               [5],  # choc
               [2]]) # milk


n = 100 # number of batches 

print('Cost per batch by cholocate bar:\n', A*B)
print('\nCost of',n,'cherry bars =',(A*B*n)[0])
print('Cost of',n,'almond bars =',(A*B*n)[1])
print('Cost of',n,'raisin bars =',(A*B*n)[2])
print('\nTotal cost for',n,'batches from supplier K =', np.sum(A*B*n))

Cost per batch by cholocate bar:
 [[60]
 [40]
 [52]]

Cost of 100 cherry bars = [[6000]]
Cost of 100 almond bars = [[4000]]
Cost of 100 raisin bars = [[5200]]

Total cost for 100 batches from supplier K = 15200


# Determining independent variables values given dependent outcomes 

Question 1

Welsh-Ryan Arena seats 15,000 people. Courtside seats cost 8 dollars, first level seats cost 6 dollars, and upper deck seats cost 4 dollars. The total revenue for a sellout is 76,000 dollars. If half the courtside seats, half the upper deck seats, and all the first level seats are sold, then the total revenue is 44,000 dollars. How many of each type of seat are there? 

In [4]:
# declare lists 
total_seats      = [1,1,1]
sell_out_revenue = [8,6,4]
# scenario_revenue represents the second scenario stated.# couryard seat costs and upper deck seat costs are divided by 2
# to respresent "half" of their seats being sold.
scenario_revenue = [4,6,2]

# use np.matrix(fn) and np.linalg(fn) to create matrix and solve for constants 
coefficients = np.matrix([total_seats,
                          sell_out_revenue,
                          scenario_revenue])
constants = np.array([15000,76000,44000])
intersection = np.linalg.solve(coefficients,constants)

intersection

array([ 3000.,  2000., 10000.])

Question 2

Vanessa invests a total of 17,500 dollars in three products. She invests one part in a mutual fund which has an annual return of 11 percent. She invests the second part in government bonds at 7 percent per year. The third part she puts in CDs at 5 percent per year. She invests twice as much in the mutual fund as in the CDs. In the first year Vanessa’s investments bring a total return of 1,495 dollars. How much did she invest in each product?

In [20]:
total_investment = [1,1,1]
total_return     = [0.11,0.07,0.05]
mf_cd_difr       = [-1,0,2]

# use np.matrix(fn) and np.linalg(fn) to create matrix and solve for constants 
coefficients = np.matrix([total_investment,
                          total_return,
                          mf_cd_difr])
constants = np.array([17500,1495,0])
intersection = np.linalg.solve(coefficients,constants)

intersection

array([9000., 4000., 4500.])

# Maximization 

Due to new environmental restrictions, Hoch Industries must use a new process to reduce pollution. The old process emits 6 g of Sulphur and 3 g of lead per liter of chemical made. The new process emits 2 g of Sulphur and 4 g of lead per liter of chemical made. The company makes a profit of 25¢ per liter under the old process and 16¢ per liter under the new process. No more than 18,000 g of Sulphur and no more than 12,000 g of lead can be emitted daily. How many liters of chemicals should be made daily under each process to maximize profits?  What is the maximum profit?

In [5]:
# declare lists: old, new  
profitability  = [0.25,0.16]
sulfur         = [6,2]
lead           = [3,4]

# use np.matrix(fn) and np.linalg(fn) to create matrix and solve for constants 
coefficients = np.matrix([sulfur,
                          lead])
constants = np.array([18000,12000])
intersection = np.linalg.solve(coefficients,constants)

profit     = intersection[0]*profitability[0]+intersection[1]*profitability[1]

print('Intersection of maximum profit \nLiters from old process:','{:.2f}'.format(intersection[0]),'\nLiters from new process:',
       intersection[1],'\nTotal profit at intersection:', '{:.2f}'.format(profit))

Intersection of maximum profit 
Liters from old process: 2666.67 
Liters from new process: 1000.0 
Total profit at intersection: 826.67


# Linear algrebra system of equations with non-squared matrices 
### Linear Programming Snp.linalg.solv - solve for x, y, z
An airline with two types of airplanes, Upper P1 and Upper P2, has contracted with a tour group to provide transportation for a minimum of 400 first class,  750 tourist class, and 1500 economy class passengers. For a certain trip, airplane Upper P 1 costs 10000 to operate and can accommodate 20 first class, 50 tourist class, and 110 economy class passengers. Airplane Upper P 2 costs 8500 to operate and can accommodate 18 first class, 30 tourist class comma and 44 economy class passengers. How many of each type of airplane should be used in order to minimize the operating cost? 

In [21]:
x = Symbol('x')
y = Symbol('y')

w = 10000*x + 8500*y                 # objective function to minimize
w_fn = lambdify(['x','y'],w,'numpy') # create a usable function from input
w

10000*x + 8500*y

In [22]:
# specify coefficients and constants
coefficients = np.array([[20,18],
                         [50,30],
                         [110,44]])
constants    = np.array([400,750,1500])

# because the matrix is not squared, each corner is evaluated separately 
intersection1 = np.linalg.solve(coefficients[0:2],constants[0:2])
intersection2 = np.linalg.solve([coefficients[0],coefficients[2]],[constants[0],constants[2]])
intersection3 = np.linalg.solve(coefficients[1:3],constants[1:3])

In [23]:
# here we are creating each possible value - rounded up or down to each which are absolute max and mins

corner_points_t = []

for i in range(len(constants)):
    
    ls = [intersection1,intersection2,intersection3]
    
    for i in enumerate(ls[i]):
        if i[0] ==0:
            floor_x = math.floor(i[1])
            ceil_x  = math.ceil(i[1])
        else: 
            floor_y = math.floor(i[1])
            ceil_y  = math.ceil(i[1])
    
    #append all four combinations to list
    corner_points_t.append([floor_x,floor_y])
    corner_points_t.append([floor_x,ceil_y])
    corner_points_t.append([ceil_x,floor_y])
    corner_points_t.append([ceil_x,ceil_y])
    
corner_points_t

[[4, 16],
 [4, 17],
 [5, 16],
 [5, 17],
 [8, 12],
 [8, 13],
 [9, 12],
 [9, 13],
 [10, 6],
 [10, 7],
 [11, 6],
 [11, 7]]

In [24]:
# create x and y lists from rounded figures
x = []
y = []

for i in enumerate(corner_points_t):
    x_t = i[1][0]
    y_t = i[1][1]
    
    x.append(x_t)
    y.append(y_t)

In [25]:
# reshape for lambified equation input
x = np.array(x).reshape(len(x),1)
y = np.array(y).reshape(len(y),1)

# pump reshaped arrays through lambified equation
linear_output = w_fn(x,y)
linear_output

array([[176000],
       [184500],
       [186000],
       [194500],
       [182000],
       [190500],
       [192000],
       [200500],
       [151000],
       [159500],
       [161000],
       [169500]])

In [26]:
# detetmine minimum & maximum values from corner points
min_index = np.where(np.in1d(linear_output,min(linear_output)))[0][0]
max_index = np.where(np.in1d(linear_output,max(linear_output)))[0][0]

In [27]:
# apply index to corner points to identify min and max x/y 
minimize = corner_points_t[min_index]
maximize = corner_points_t[max_index]

In [28]:
# print out mins and maxes
print('Minimization occurs at:','\nx =',round(minimize[0],1),'y =',round(minimize[1],1),'w =',int(min(linear_output)))
print('Maximization occurs at:','\nx =',round(maximize[0],1),'y =',round(maximize[1],1),'z =',int(max(linear_output)))

Minimization occurs at: 
x = 10 y = 6 w = 151000
Maximization occurs at: 
x = 9 y = 13 z = 200500


In [29]:
# print all rounded possibilities
stack = np.hstack((corner_points_t,linear_output.reshape(len(x),1)))
stack

array([[     4,     16, 176000],
       [     4,     17, 184500],
       [     5,     16, 186000],
       [     5,     17, 194500],
       [     8,     12, 182000],
       [     8,     13, 190500],
       [     9,     12, 192000],
       [     9,     13, 200500],
       [    10,      6, 151000],
       [    10,      7, 159500],
       [    11,      6, 161000],
       [    11,      7, 169500]])

In [30]:
# minimization --> change the >= to <= for maximization
validation = []

for i in stack:
    x_y  = np.array([i[0],i[1]])  # obtain x & y from array
    const = x_y * coefficients    # multiply x & y values by coefficients 
    
    test_array = np.hstack((const,constants.reshape(len(constants),1)))
    
    if test_array[0][0]+test_array[0][1] >= test_array[0][2]:
        test1 = True
    else:
        test1 = False
    if test_array[1][0]+test_array[1][1] >= test_array[1][2]:
        test2 = True
    else:
        test2 = False
    if test_array[2][0]+test_array[2][1] >= test_array[2][2]:
        test3 = True
    else:
        test3 = False
        
    if test1 == True and test2 == True and test3 == True:
        validation.append(i)
    else:
        pass

In [31]:
validation

[array([     9,     13, 200500])]

### Optimization using scipy.optimize import linprog

#### Question 1
If 30 lb of rice and 20 lb of potatoes cost 21.20, and 20 lb of rice and 12 lb of potatoes 
cost 13.52, how much will 10 lb of rice and 50 lb of potatoes cost?

In [3]:
coefficients = np.array([[30,20],[20,12]])
constants = np.array([21.2,13.52])
variables = np.linalg.solve(coefficients,constants)

print(f'The variables are: {variables}.')

solution = 10*variables[0] + 50*variables[1]

print(f'The total cost is: {solution}.')

The variables are: [0.4  0.46].
The total cost is: 26.999999999999986.


#### Question 2

A person has 42,000 invested in stock A and stock B. Stock A currently sells 
for 20 a share and stock B sells for 80 a share. If stock B triples in value 
and stock A goes up 50_percent, his stock will be worth 111000. How many shares of 
each stock does he own?

In [ ]:
coefficients = np.array([[20,80],[30,240]])
constants = np.array([42000,111000])
variables = np.linalg.solve(coefficients,constants)

print(f'The variables are: {variables}.')

In [32]:
# Maximization Function
z = [-1,-1,-1]
# Coefficients
a = [[120,130,150],
     [160,90,90],
     [4.97,4.45,4.65]]
# Constants
b = [31200,30000,500]

method = 'simplex'

res = linprog(z, A_ub=a, b_ub=b,bounds=(0, None))

print("Status:", res.message, '\n\nMaximum:',
     -1*res.fun,
     "\nOptimial x1:", round(res.x[0]),
     "\nOptimial x2:", round(res.x[1]),
     "\nOptimial x3:", round(res.x[2]))

Status: Optimization terminated successfully. 

Maximum: 112.35955056172406 
Optimial x1: 0.0 
Optimial x2: 112.0 
Optimial x3: 0.0


### Optimization using Pulp

#### Minimization

To be at his best as an athlete, Jimmy needs at least 10 units of vitamin A, 12 units of vitamin B, and 20 units of vitamin C per day. Pill 1 contains 4 units of A and 3 of B. Pill 2 contains 1 unit of A, 2 of B, and 4 of C. Pill 3 contains 10 units of A, 1 of B, and 5 of C. Pill 1 costs 6 cents, pill 2 costs 8 cents, and pill 3 costs 1 cent. How many of each pill must Jimmy take to minimize his cost, and what is that cost?

In [7]:
# Pulp
x1 = pulp.LpVariable("x1", 0, None) # x1>=0
x2 = pulp.LpVariable("x2", 0, None) # x2>=0
x3 = pulp.LpVariable("x3", 0, None) # x3>=0

# defines the problem
prob = pulp.LpProblem("problem", pulp.LpMinimize)

# defines the constraints
prob += 4*x1 + 1*x2 + 10*x3 >= 10
prob += 3*x1 + 2*x2 + 1*x3  >= 12
prob += 0*x1 + 4*x2 + 5*x3  >= 20

# defines the objective function to maximize
prob += 6*x1 + 8*x2 + 1*x3

# solve the problem
status = prob.solve()
pulp.LpStatus[status]

# print the results
print("Billy's minimized cost is 12 cents to obtain his minimal nutrition. He will take:",
       '\n Pill 1:',pulp.value(x1),
       '\n Pill 2:',pulp.value(x2),
       '\n Pill 3:',pulp.value(x3))

Billy's minimized cost is 12 cents to obtain his minimal nutrition. He will take: 
 Pill 1: 0.0 
 Pill 2: 0.0 
 Pill 3: 12.0


#### Maximization

Better Buy stocks Blu-ray DVD players, surround sound systems, and Smart TVs. They have limited storage space and can stock a maximum of 210 of these three machines. They know from past experience that they should stock twice as many DVD players as stereo systems and at least 30 TVs. If each DVD player sells for 450, each surround sound system sells for 2000, and each TV sells for 750, how many of each should be stocked and sold for maximum revenues? What is the maximum revenue?

In [8]:
# Pulp
x1 = pulp.LpVariable("x1", 0, None) # x1>=0
x2 = pulp.LpVariable("x2", 0, None) # x2>=0
x3 = pulp.LpVariable("x3", 0, None) # x3>=0

# defines the problem
prob = pulp.LpProblem("problem", pulp.LpMaximize)

# defines the constraints
prob += 1*x1 + 1*x2 + 1*x3 <= 210
prob += 0*x1 + 0*x2 + 1*x3 >= 30
prob += 1*x1 - 2*x2 + 0*x3 >= 0

# defines the objective function to maximize
prob += 450*x1 + 2000*x2 + 750*x3

# solve the problem
status = prob.solve()
pulp.LpStatus[status]

z = 450*pulp.value(x1) + 2000*pulp.value(x2) + 750*pulp.value(x3)

# print the results
print("Better Buy's maximum revenue is",z,  
       '\nBlu-ray DVDs sold:',pulp.value(x1),
       '\nStereo Systems sold:',pulp.value(x2),
       '\nSmart TVs sold:',pulp.value(x3))

Better Buy's maximum revenue is 196500.0 
Blu-ray DVDs sold: 120.0 
Stereo Systems sold: 60.0 
Smart TVs sold: 30.0
